In [ ]:
import pygame
import random
import math

pygame.init()

width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Elastic Collision Simulation')

WHITE = (255, 255, 255)
GRAY = (127, 127, 127)
BLACK = (0, 0, 0)

class Particle:
    def __init__(self, x, y, speed):
        self.position = pygame.math.Vector2(x, y)
        self.velocity = pygame.math.Vector2(random.uniform(-1, 1), random.uniform(-1, 1)).normalize() * speed
        self.acceleration = pygame.math.Vector2(0, 0)
        self.mass = random.uniform(1, 20)
        self.radius = math.sqrt(self.mass) * 10

    def apply_force(self, force):
        f = force / self.mass
        self.acceleration += f

    def update(self):
        self.velocity += self.acceleration
        self.position += self.velocity
        self.acceleration *= 0

    def collide(self, other):
        impact_vector = other.position - self.position
        distance = impact_vector.length()
        if distance < self.radius + other.radius:
            overlap = self.radius + other.radius - distance
            direction = impact_vector.normalize()
            self.position -= direction * (overlap / 2)
            other.position += direction * (overlap / 2)

            normal = (self.position - other.position).normalize()
            tangent = pygame.math.Vector2(-normal.y, normal.x)
            
            dpTan1 = self.velocity.dot(tangent)
            dpTan2 = other.velocity.dot(tangent)
            
            dpNorm1 = self.velocity.dot(normal)
            dpNorm2 = other.velocity.dot(normal)
            
            m1 = (dpNorm1 * (self.mass - other.mass) + 2 * other.mass * dpNorm2) / (self.mass + other.mass)
            m2 = (dpNorm2 * (other.mass - self.mass) + 2 * self.mass * dpNorm1) / (self.mass + other.mass)
            
            self.velocity = tangent * dpTan1 + normal * m1
            other.velocity = tangent * dpTan2 + normal * m2

    def edges(self):
        if self.position.x > width - self.radius:
            self.position.x = width - self.radius
            self.velocity.x *= -1
        elif self.position.x < self.radius:
            self.position.x = self.radius
            self.velocity.x *= -1

        if self.position.y > height - self.radius:
            self.position.y = height - self.radius
            self.velocity.y *= -1
        elif self.position.y < self.radius:
            self.position.y = self.radius
            self.velocity.y *= -1

    def show(self):
        pygame.draw.circle(screen, WHITE, (int(self.position.x), int(self.position.y)), int(self.radius))

def create_particles(num_particles, speed):
    particles = []
    for _ in range(num_particles):
        x = random.uniform(50, width - 50)
        y = random.uniform(50, height - 50)
        particles.append(Particle(x, y, speed))
    return particles

num_particles = 10
particle_speed = 3

particles = create_particles(num_particles, particle_speed)

running = True
clock = pygame.time.Clock()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill(BLACK)

    for i in range(num_particles):
        for j in range(i + 1, num_particles):
            particles[i].collide(particles[j])

    for particle in particles:
        particle.update()
        particle.edges()
        particle.show()

    pygame.display.flip()
    clock.tick(60)

pygame.quit()